<a href="https://colab.research.google.com/github/Harta-Tahta-Data/HTD-Compfest-ML/blob/main/AIC_EsAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import operator
import re
import ast

from matplotlib import pyplot as plt

import seaborn as sns
import urllib
from ast import literal_eval
from collections import Counter

from gensim.models import Word2Vec

from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import TimeDistributed
from keras.layers import SimpleRNN, RNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# url


In [2]:
training_dataset = 'https://raw.githubusercontent.com/Harta-Tahta-Data/HTD-Compfest-ML/main/training_dataset.csv'
training_dataset_6 = 'https://raw.githubusercontent.com/Harta-Tahta-Data/HTD-Compfest-ML/main/training_dataset_6.csv'
id_50 = 'https://raw.githubusercontent.com/Harta-Tahta-Data/HTD-Compfest-ML/main/id_50k%20(1).txt'
kata_dataset = 'https://raw.githubusercontent.com/Harta-Tahta-Data/HTD-Compfest-ML/main/110k_kata.csv'
kata_baku = 'https://raw.githubusercontent.com/Harta-Tahta-Data/HTD-Compfest-ML/main/kata_baku.csv'
kata_tidak_baku = 'https://raw.githubusercontent.com/Harta-Tahta-Data/HTD-Compfest-ML/main/kata_tidak_baku.csv'
slang = 'https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt'

In [3]:
# import id_50k.txt
files = urllib.request.urlopen(id_50)
file_nama = open('id_50k.txt', 'w')
for line in files:
  decoded_line = line.decode('latin-1')
  file_nama.write(decoded_line)
file_nama.close()

# import kata_tidak_baku
kata_tidak_baku = pd.read_csv(kata_tidak_baku)
kata_baku = pd.read_csv(kata_baku)

In [4]:
# # # import kata_tidak _baku dan kata_baku 
# kata_baku = 'https://raw.githubusercontent.com/Harta-Tahta-Data/HTD-Compfest-ML/main/kata_baku.txt'
# kata_tidak_baku = 'https://raw.githubusercontent.com/Harta-Tahta-Data/HTD-Compfest-ML/main/kata_tidak_baku.txt'

# import ast
# from ast import literal_eval
# import urllib
# from urllib import request
# files = urllib.request.urlopen(kata_baku)
# file_baku = open('kata_baku.txt', 'w')
# for line in files:
#   decoded_line = line.decode('latin-1')
#   file_baku.write(decoded_line)

# file_baku = open('kata_baku.txt', 'r')
# kata_baku = {'kata': []}
# for line in file_baku:
#   if line == 'kata':
#     continue
#   else:
#     kata_baku['kata'].append(line)

# files = urllib.request.urlopen(kata_tidak_baku)
# file_tidak_baku = open('kata_tidak_baku.txt', 'w')
# for line in files:
#   decoded_line = line.decode('latin-1')
#   file_tidak_baku.write(decoded_line)

# file_tidak_baku = open('kata_tidak_baku.txt', 'r')
# kata_tidak_baku = {'tidak_baku':[], 'baku':[]}
# for line in file_tidak_baku:
#   katasiapa = line.split(',')
#   if 'tidak_baku' in katasiapa and 'baku' in katasiapa:
#     continue
#   else:
#     kata_tidak_baku['tidak_baku'].append(katasiapa[0])
#     kata_tidak_baku['baku'].append(katasiapa[1])

In [5]:
print(len(kata_baku['kata']))
print(len(kata_tidak_baku['tidak_baku']))
print(len(kata_tidak_baku['baku']))

99670
4776
4776


# Load & preprocess dataset

In [6]:
pos_tags=pd.read_csv(training_dataset)

# fix typo in the dataset
pos_tags = pos_tags.replace("fw", "FW")

In [7]:
# label diurut berdasarkan jumlah POS tag di dataset (descending)
labels_sorted = {}

for i in range(len(pos_tags)):
  if pos_tags["pos_tag"][i] not in labels_sorted:
    labels_sorted[pos_tags["pos_tag"][i]] = 1
  else:
    labels_sorted[pos_tags["pos_tag"][i]] += 1

marklist = sorted(labels_sorted.items(), key=operator.itemgetter(1), reverse=True) 
labels_sorted=dict(marklist)

# 23 classes
label_list = list(labels_sorted.keys())

# id + class
labels = pd.DataFrame({"POS label": label_list})
labels.index += 1

In [8]:
label_list

['NN',
 'NNP',
 'VB',
 'Z',
 'IN',
 'CD',
 'SC',
 'JJ',
 'PRP',
 'CC',
 'MD',
 'PR',
 'RB',
 'FW',
 'SYM',
 'NEG',
 'NND',
 'OD',
 'DT',
 'X',
 'WH',
 'RP',
 'UH']

In [9]:
# load dataset
sentence_pos_tags=pd.read_csv(training_dataset_6)

# set X and Y
X = [literal_eval(i) for i in sentence_pos_tags["sentences"]]
Y = [literal_eval(i) for i in sentence_pos_tags["pos_tag"]]
X_sentences = X.copy()

# Encoding X and Y

In [10]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X)
X_encoded = word_tokenizer.texts_to_sequences(X)

tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(Y)
Y_encoded = tag_tokenizer.texts_to_sequences(Y)

In [11]:

lengths = [len(seq) for seq in X_encoded]

MAX_SEQ_LENGTH = max(lengths)  # sequences with length larger than 105 will be truncated

X_padded = pad_sequences(X_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")
Y_padded = pad_sequences(Y_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")

# assign padded sequences to X and Y
X, Y = X_padded, Y_padded

# Word embedding

In [12]:
# define model
model = Word2Vec(sentences=X_sentences)

In [13]:
# hyperparameters
EMBEDDING_SIZE  = model.wv.vector_size                  # 100
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1    # 17124

In [14]:
# create an empty embedding matix
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))

# create a word to index dictionary mapping
word2id = word_tokenizer.word_index

# copy vectors from word2vec model to the words present in corpus
for word, index in word2id.items():
    try:
        embedding_weights[index, :] = model.wv[word]
    except KeyError:
        pass

Y = to_categorical(Y)

In [15]:
# save
model.wv.save_word2vec_format("custom_word_embedding_id.txt")

# Split data

In [16]:
TEST_SIZE = 0.15
VALID_SIZE = 0.15

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=4)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=VALID_SIZE, random_state=4)

# Training

## Helper functions

In [18]:
# plot training
def plot_training(training_model):
    plt.plot(training_model.history["acc"])
    plt.plot(training_model.history["val_acc"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "test"], loc="lower right")
    plt.show()

In [19]:
# preprocess kalimat
def split_kalimat(kalimat):
    kalimat_baru = []
    kalimat = re.split(r"(\W+)", kalimat)

    while "" in kalimat:
        kalimat.remove("")

    while " " in kalimat:
        kalimat.remove(" ")

    kalimat = [x.strip(" ") for x in kalimat]

    for x in kalimat:
        x = x.split(" ")
        for i in x:
            kalimat_baru.append(i)

    return kalimat_baru

## RNN model (w/ weights from the word embedding)

In [20]:
# hyperparameters
EMBEDDING_SIZE  = model.wv.vector_size                  # 100
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1    # 17124
NUM_CLASSES = Y.shape[2]

In [21]:
rnn_model = Sequential()

rnn_model.add(Embedding(input_dim=VOCABULARY_SIZE,
                        output_dim=EMBEDDING_SIZE,
                        input_length=MAX_SEQ_LENGTH,
                        weights=[embedding_weights],
                        trainable=True))

rnn_model.add(SimpleRNN(64, return_sequences=True))

rnn_model.add(TimeDistributed(Dense(NUM_CLASSES, activation="softmax")))

In [22]:
rnn_model.compile(loss="categorical_crossentropy",
                   optimizer="adam",
                   metrics=["acc"])

In [23]:
rnn_training = rnn_model.fit(X_train,
                             Y_train,
                             batch_size=128,
                             epochs=10,
                             validation_data=(X_validation, Y_validation))

Epoch 1/10
60/60 [==============================] - 7s 99ms/step - loss: 1.1314 - acc: 0.7757 - val_loss: 0.5327 - val_acc: 0.8651
Epoch 2/10
60/60 [==============================] - 6s 95ms/step - loss: 0.4189 - acc: 0.8994 - val_loss: 0.3302 - val_acc: 0.9271
Epoch 3/10
60/60 [==============================] - 6s 94ms/step - loss: 0.2478 - acc: 0.9494 - val_loss: 0.1942 - val_acc: 0.9616
Epoch 4/10
60/60 [==============================] - 6s 93ms/step - loss: 0.1433 - acc: 0.9731 - val_loss: 0.1230 - val_acc: 0.9753
Epoch 5/10
60/60 [==============================] - 6s 94ms/step - loss: 0.0911 - acc: 0.9820 - val_loss: 0.0904 - val_acc: 0.9803
Epoch 6/10
60/60 [==============================] - 6s 94ms/step - loss: 0.0655 - acc: 0.9863 - val_loss: 0.0739 - val_acc: 0.9830
Epoch 7/10
60/60 [==============================] - 6s 95ms/step - loss: 0.0515 - acc: 0.9888 - val_loss: 0.0647 - val_acc: 0.9843
Epoch 8/10
60/60 [==============================] - 6s 94ms/step - loss: 0.0427 - a

In [24]:
# evaluate accuracy on test set
loss, accuracy = rnn_model.evaluate(X_test, Y_test, verbose=1)
print("Loss: {0},\nAccuracy: {1}".format(loss, accuracy))

50/50 [==============================] - 1s 11ms/step - loss: 0.0513 - acc: 0.9862
Loss: 0.051318634301424026,
Accuracy: 0.9862321019172668


In [25]:
rnn_model.save('model_v6.h5')

In [26]:
import pickle

# saving
with open('tokenizer_v6.pickle', 'wb') as handle:
    pickle.dump(word_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# POS tagger

In [27]:
# prediksi POS tag kalimat
def prediksi_pos(kalimat):
    kalimat_encoded = word_tokenizer.texts_to_sequences([kalimat])
    kalimat_padded = pad_sequences(kalimat_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")

    pred_kalimat = rnn_model.predict(kalimat_padded[0])

    result = []
    for i in range(len(pred_kalimat)):
        id = np.argmax(pred_kalimat[i][0])
        if id != 0:
            result.append(label_list[id-1])

    # unknown words detected
    if len(result) != len(kalimat):
        for id, i in enumerate(kalimat):
            word_encoded = word_tokenizer.texts_to_sequences([[i]])
            word_padded = pad_sequences(word_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")
            if np.count_nonzero(word_padded) == 0:
                result.insert(id, "X")
    
    return result

In [28]:
print(MAX_SEQ_LENGTH)

105


# Typo checker

## Dataset: id_50k.txt
[Sumber](https://github.com/hermitdave/FrequencyWords/blob/master/content/2018/id/id_50k.txt)

In [29]:
import ast
from ast import literal_eval
# open/import slang words dataset
#slang = open("combined_slang_words.txt", "r")
#contents = slang.read()
#slang_dict = ast.literal_eval(contents)
#slang.close()
files = urllib.request.urlopen(slang)
file_slang = open('combined_slang_words.txt', 'w')
for line in files:
  decoded_line = line.decode('latin-1')
  file_slang.write(decoded_line)

file_slang = open('combined_slang_words.txt', 'r')
contents = file_slang.read()
slang_dict = ast.literal_eval(contents)
file_slang.close()

In [30]:
# http://norvig.com/spell-correct.html

def words(text):
    words = []
    lines = text.split("\n")
    for i, line in enumerate(lines):
        word = lines[i].split(" ")[0]
        words.append(word)
    return words

WORDS = words(open('id_50k.txt').read())

def P(word): 
    "Probability of `word`."
    word_counts = []
    lines = open('id_50k.txt').read().split("\n")
    for i, line in enumerate(lines):
        word_count = lines[i].split(" ")[1]
        word_counts.append(word_count)
    word_prob = word_counts[WORDS.index(word)]
    return int(word_prob)

def correction(word): 
    "Most probable spelling correction for word."
    try:
        if word in slang_dict:
            max_word = slang_dict[word]
        else:
            max_word = max(candidates(word), key=P)
    except ValueError:
        max_word = word
    return max_word

def candidates(word): 
    "Generate possible spelling corrections for word."
    candidates = (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
    new_candidates = candidates.copy()
    word_counts = []
    lines = open('id_50k.txt').read().split("\n")
    for i, line in enumerate(lines):
        word_count = lines[i].split(" ")[0]
        word_counts.append(word_count)

    for i in candidates:
        # csv

        if i in kata_tidak_baku['tidak_baku'].values:
            new_candidates.remove(i)
            new_word = kata_tidak_baku['baku'][kata_tidak_baku.loc[kata_tidak_baku['tidak_baku'] == i].index[0]]
            # print(new_word, type(new_word))
            if new_word in word_counts:
                new_candidates.add(new_word)
        elif i not in kata_baku['kata'].values:
            new_candidates.remove(i)
        
        #txt

        # if i in kata_tidak_baku['tidak_baku']:
        #     new_candidates.remove(i)
        #     new_word = kata_tidak_baku['baku'][kata_tidak_baku['tidak_baku'].index(i)]
        #     print(new_word, type(new_word))
        #     if new_word in word_counts:
        #       new_candidates.add(new_word)
        # elif i not in kata_baku['kata']:
        #     new_candidates.remove(i)

    return new_candidates

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [31]:
candidates('apotik')

{'apotek'}

In [32]:
if 'apotik' in kata_tidak_baku['tidak_baku']:
  print(True)
kata_tidak_baku.loc[kata_tidak_baku['tidak_baku'] == 'apotik']

,tidak_baku,baku
97,apotik,apotek


# Grammar checker (rules)

## Sebelum POS tag

In [33]:
di = ['dia', 'diabetes', 'diad', 'diadem', 'diafon', 'diaforetik', 'diafragma', 'diagenesis', 'diagnosis', 'diagnostik', 'diagometer', 'diagonal', 'diagram', 'diaken', 'diakon', 'diakones', 'diakonia', 'diakritik', 'diakronis', 'dialek', 'dialektal', 'dialektik', 'dialektika', 'dialektis', 'dialektologi', 'dialinguistik', 'dialisis', 'dialog', 'dialogis', 'diam', 'diamagnetisme', 'diameter', 'diametral', 'diamorf', 'dian', 'diang', 'diaper', 'diapositif', 'diar', 'diare', 'dias', 'diasistem', 'diaspora', 'diastase', 'diastole', 'diat', 'diaterman', 'diatermi', 'diatermik', 'diatesis', 'diatipe', 'diatom', 'diatomit', 'diatonik', 'diatopik', 'diayah', 'dibasa', 'didaktik', 'didaktikus', 'didaktis', 'didih', 'didik', 'didis', 'didong', 'dielektrik', 'diensefalon', 'dies natalis', 'diesel', 'diet', 'dietetika', 'difabel', 'diferensial', 'diferensiasi', 'difluens', 'difluensi', 'difraksi', 'difteri', 'diftong', 'difusi', 'digdaya', 'digenesis', 'digestif', 'digit', 'digital', 'digitalin', 'digitalis', 'digitalisasi', 'diglosia', 'digraf', 'digresi', 'Digul', 'dihedral', 'dihidroksil', 'dik', 'dikara', 'dikau', 'dikit', 'diklorida', 'dikotil', 'dikotomi', 'dikroisme', 'dikromat', 'dikromatik', 'diksa', 'diksi', 'diktat', 'diktator', 'diktatorial', 'diktatoris', 'dikte', 'diktum', 'dil', 'dila', 'dilak', 'dilam', 'dilasi', 'dilatasi', 'dilatometer', 'dilema', 'dilematik', 'diler', 'diletan', 'diluvium', 'dim', 'dimensi', 'dimer', 'diminutif', 'dimorfik', 'dimorfisme', 'din', 'dina', 'dinamik', 'dinamika', 'dinamis', 'dinamisator', 'dinamisme', 'dinamit', 'dinamo', 'dinamometer', 'dinar', 'dinasti', 'dinding', 'dingin', 'dingkis', 'dingkit', 'dingklang', 'dingklik', 'dingo', 'dini', 'diniah', 'dinosaurus', 'diode', 'dioesis', 'dioksida', 'dioksin', 'diopsida', 'dioptase', 'dioptri', 'diorama', 'diorit', 'dioses', 'dipan', 'diplo', 'diploid', 'diploma', 'diplomasi', 'diplomat', 'diplomatik', 'diplomatis', 'dipsomania', 'diptera', 'diptotos', 'dirah', 'diraja', 'direk', 'direksi', 'direktorat', 'direktorium', 'direktris', 'direktur', 'dirgahayu', 'dirgantara', 'dirham', 'diri', 'dirigen', 'diris', 'dirus', 'disagio', 'disakarida', 'disastria', 'disbursemen', 'disdrometer', 'disekuilibrium', 'disensus', 'disentri', 'disertasi', 'disfonia', 'disfungsi', 'disharmoni', 'disiden', 'disilabik', 'disimilasi', 'disinfektan', 'disinformasi', 'disinsentif', 'disintegrasi', 'disiplin', 'disjoki', 'disjungsi', 'disjungtif', 'diska', 'disket', 'diskiasis', 'disklimaks', 'disko', 'diskoid', 'diskon', 'diskontinu', 'diskontinuitas', 'diskonto', 'diskordans', 'diskorobik', 'diskotek', 'diskredit', 'diskrepansi', 'diskresi', 'diskriminasi', 'diskriminatif', 'diskriminator', 'diskualifikasi', 'diskulpasi', 'diskursif', 'diskus', 'diskusi', 'dislalia', 'disleksia', 'dislokasi', 'dismembrasio', 'dismenorea', 'dismutasi', 'disolventia', 'disonansi', 'disoperasi', 'disorder', 'disorganisasi', 'disorientasi', 'disosiasi', 'dispareunia', 'disparitas', 'dispensasi', 'dispenser', 'dispepsia', 'dispersal', 'dispersi', 'disposisi', 'disposotio', 'disprosium', 'disrupsi', 'distabilitas', 'distal', 'distansi', 'distikiasis', 'distikon', 'distilasi', 'distilator', 'distingsi', 'distingtif', 'distoma', 'distorsi', 'distosia', 'distribusi', 'distributor', 'distrik', 'disuasi', 'disuria', 'dito', 'ditransitif', 'diuresis', 'diuretik', 'diurnal', 'divergen', 'divergensi', 'diversifikasi', 'diversitas', 'divestasi', 'dividen', 'divisi']
ke = ['kebab', 'kebabal', 'kebah', 'kebaji', 'kebal', 'kebam', 'kebas', 'kebat', 'kebaya', 'kebayan', 'kebel', 'kebelet', 'kebembem', 'kebin', 'kebiri', 'keblangsak', 'keblinger', 'kebuk', 'kebul', 'kebuli', 'kebun', 'kebur', 'kebut', 'kebyar', 'kecai', 'kecak', 'kecak pinggang', 'kecalingan', 'kecam', 'kecambah', 'kecamuk', 'kecandan', 'kecap', 'kecapi', 'kecar', 'kece', 'kecebong', 'kecek', 'kecele', 'keceng', 'kecepek', 'kecer', 'kecewa', 'keci', 'keciak', 'kecibak', 'kecibeling', 'kecik', 'kecil', 'kecil hati', 'kecimik', 'kecimpring', 'kecimpung', 'kecimus', 'kecipak', 'kecipir', 'kecipuk', 'kecit', 'keciut', 'kecoak', 'kecoh', 'kecombrang', 'kecong', 'kecrek', 'kecu', 'kecuali', 'kecubung', 'kecumik', 'kecup', 'kecut', 'kedabu', 'kedadak', 'kedah', 'kedai', 'kedal', 'kedaluwarsa', 'kedam', 'kedang', 'kedangkai', 'kedangkan', 'kedangsa', 'kedap', 'kedasih', 'kedau', 'kedaung', 'kedayan', 'kedebong', 'kedek', 'kedekai', 'kedeki', 'kedekut', 'kedelai', 'kedele', 'kedemplung', 'kedempung', 'kedengkang', 'keder', 'kedera', 'kederang', 'kedewaga', 'kedi', 'kedidi', 'kedik', 'kedikit', 'kedip', 'kedok', 'kedondong', 'kedongkok', 'kedot', 'kedua', 'keduanya', 'keduduk', 'keduk', 'kedul', 'kedumung', 'kedung', 'kedut', 'keferdom', 'kehel', 'keibodan', 'kejai', 'kejam', 'kejamas', 'kejan', 'kejang', 'kejap', 'kejar', 'kejat', 'kejawen', 'kejen', 'kejer', 'keji', 'kejip', 'Kejora', 'keju', 'kejuju', 'kejur', 'kejut', 'kek', 'kekah', 'kekal', 'kekam', 'kekandi', 'kekang', 'kekapas', 'kekar', 'kekara', 'kekas', 'kekat', 'kekau', 'kekawin', 'kekeba', 'kekebik', 'kekeh', 'kekek', 'kekel', 'kekemben', 'kekep', 'keker', 'keki', 'kekitir', 'kekok', 'kekol', 'kelab', 'kelabak', 'kelabang', 'kelabat', 'kelabau', 'kelabu', 'keladak', 'keladan', 'keladau', 'keladi', 'kelah', 'kelahi', 'kelai', 'kelak', 'kelakah', 'kelakanji', 'kelakar', 'kelalang', 'kelam', 'kelambai', 'kelambir', 'kelambit', 'kelambu', 'kelambur', 'kelamin', 'kelamkari', 'kelana', 'kelang', 'kelangkan', 'kelangkang', 'kelanjar', 'kelantang', 'kelapa', 'kelar', 'kelara', 'kelarah', 'kelarai', 'kelaras', 'kelari', 'kelas', 'kelasa', 'kelasah', 'kelasak', 'kelasi', 'kelat', 'kelati', 'kelawan', 'kelayan', 'kelayang', 'kelayu', 'kelder', 'kelebat', 'kelebek', 'kelebet', 'kelebu', 'kelebuk', 'kelebut', 'keledai', 'keledang', 'keledar', 'keledek', 'kelejat', 'kelek', 'kelekap', 'kelekatu', 'kelelap', 'kelelawar', 'kelelesa', 'kelelot', 'kelemayar', 'kelemayuh', 'kelembahang', 'kelembak', 'kelemban', 'kelembuai', 'kelempai', 'kelemping', 'kelemur', 'kelendara', 'keleneng', 'kelengar', 'kelenggara', 'kelengkeng', 'kelengkiak', 'kelening', 'kelenjar', 'kelentang', 'kelenteng', 'kelentik', 'kelenting', 'kelentit', 'kelentong', 'kelentung', 'kelenung', 'kelenyit', 'kelep', 'kelepai', 'kelepak', 'kelepat', 'kelepek', 'kelepet', 'kelepik', 'kelepir', 'kelepit', 'kelepuk', 'kelepur', 'keler', 'kelereng', 'kelesa', 'kelesah', 'keleseh peseh', 'kelesek', 'kelesot', 'keletah', 'keletak', 'keletang', 'keletik', 'keletuk', 'keletung', 'kelewang', 'keli', 'kelian', 'keliar', 'kelibang', 'kelibat', 'kelicap', 'kelici', 'kelih', 'kelijak', 'kelik', 'kelika', 'kelikat', 'keliki', 'kelikih', 'kelikik', 'kelikir', 'keliling', 'kelilip', 'kelim', 'kelimpanan', 'kelimpungan', 'kelimun', 'kelimut', 'kelinci', 'kelincir', 'kelindan', 'kelingking', 'kelingsir', 'kelining', 'kelinjat', 'kelintang', 'kelintar', 'kelinting', 'kelip', 'kelipat', 'kelir', 'keliru', 'kelis', 'kelisera', 'kelit', 'keliti', 'kelitik', 'keliwon', 'kelobot', 'kelobotisme', 'kelocak', 'keloceh', 'kelodan', 'keloelektrovolt', 'keloid', 'kelojot', 'kelok', 'kelokak', 'kelola', 'kelolong', 'kelom', 'kelombeng', 'kelompang', 'kelompen', 'kelompok', 'kelon', 'keloneng', 'kelonet', 'kelong', 'kelongkong', 'kelongsong', 'kelontang', 'kelontong', 'kelonyo', 'kelop', 'kelopak', 'kelor', 'kelorak', 'kelos', 'kelosok', 'kelotok', 'keloyak', 'keloyang', 'keloyor', 'kelp', 'kelu', 'kelua', 'keluai', 'keluak', 'keluan', 'keluang', 'keluangsa', 'keluar', 'keluarga', 'kelubak', 'kelubi', 'keluburan', 'keluh', 'kelui', 'keluih', 'keluk', 'kelukup', 'kelukur', 'keluli', 'kelulu', 'kelulus', 'kelulut', 'kelumit', 'kelumpang', 'kelumun', 'kelun', 'keluna', 'kelunak', 'kelung', 'kelupas', 'kelupur', 'keluron', 'kelurut', 'kelus', 'kelut', 'kelutum', 'keluwung', 'keluyuk', 'keluyur', 'kemah', 'kemal', 'kemala', 'kemam', 'kemamang', 'kemang', 'kemangi', 'kemarau', 'kemari', 'kemarin', 'kemaruk', 'kemas', 'kemat', 'kematu', 'kematus', 'kemayu', 'kembal', 'kembali', 'kemban', 'kembang', 'kembang biak', 'kembar', 'kembatu', 'kembayat', 'kembeng', 'kembera', 'kembili', 'kemboja', 'kembol', 'kembu', 'kembuk', 'kembung', 'kembur', 'kembut', 'kemeja', 'kemejan', 'kemekmek', 'kemelut', 'kemenakan', 'kemendalam', 'kemendang', 'kemendur', 'kementam', 'kemenyan', 'kemerakan', 'kemesu', 'kemi', 'kemih', 'kemik', 'kemilap', 'kemiluminesens', 'kemiri', 'kemit', 'kemlaka', 'kemlandingan', 'kemoceng', 'kemokinesis', 'kemon', 'kemopsikiatri', 'kemoterapi', 'kempa', 'kempal', 'kempang', 'kempas', 'kempek', 'kempes', 'kempetai', 'kempis', 'kempit', 'kemplang', 'kempot', 'kempu', 'kempuh', 'kempul', 'kempunan', 'kempung', 'kemput', 'kempyang', 'kemu', 'kemudi', 'kemudian', 'kemukus', 'kemul', 'kemumu', 'kemuncup', 'kemung', 'kemungkus', 'kemuning', 'kemunting', 'kemurgi', 'kemut', 'kemutul', 'ken', 'kena', 'kenaf', 'kenal', 'kenan', 'kenang', 'kenanga', 'kenap', 'kenapa', 'kenapang', 'kenari', 'kenas', 'kencan', 'kencana', 'kencang', 'kencar', 'kenceng', 'kencing', 'kencit', 'kencong', 'kencreng', 'kencung', 'kencur', 'kendaga', 'kendal', 'kendala', 'kendali', 'kendana', 'kendang', 'kendara', 'kendati', 'kendayakan', 'kendeka', 'kenderi', 'kendi', 'kendil', 'kendit', 'kendo', 'kendong', 'kenduduk', 'kendung', 'kendur', 'kenduri', 'kenek', 'keneker', 'kenem', 'kenematik', 'kenes', 'keng', 'kengkang', 'kengkeng', 'kenidai', 'kenikir', 'kening', 'kenohong', 'kenong', 'kenop', 'kensel', 'kental', 'kentang', 'kentar', 'kentara', 'kenteng', 'kentrung', 'kentung', 'kentut', 'kenur', 'kenya', 'kenyal', 'kenyam', 'kenyang', 'kenyat', 'kenyi', 'kenyih', 'kenyir', 'kenyit', 'kenyut', 'keok', 'keong', 'kep', 'kepada', 'kepah', 'kepai', 'kepak', 'kepal', 'kepala', 'kepalang', 'kepam', 'kepang', 'kepar', 'keparat', 'kepayang', 'kepecong', 'kepek', 'kepel', 'kepencong', 'kepeng', 'keper', 'keperancak', 'kepet', 'kepetang', 'kepialu', 'kepiat', 'kepik', 'kepil', 'kepincut', 'kepinding', 'keping', 'kepingin', 'kepinis', 'kepinjal', 'kepiri', 'kepis', 'kepit', 'kepiting', 'keplak', 'kepleset', 'keplok', 'kepodang', 'kepoh', 'kepol', 'keponakan', 'kepot', 'keprak', 'keprek', 'kepret', 'kepris', 'kepruk', 'kepuh', 'kepuk', 'kepul', 'kepulaga', 'kepundan', 'kepundung', 'kepung', 'kepurun', 'keputren', 'kepuyuk', 'kera', 'kerabang', 'kerabat', 'kerabik', 'kerabu', 'keracak', 'keracap', 'keraeng', 'kerah', 'kerahi', 'kerai', 'kerajang', 'kerajat', 'kerak', 'kerakah', 'kerakal', 'kerakap', 'kerakeling', 'keram', 'kerama', 'keraman', 'keramas', 'keramat', 'keramba', 'kerambil', 'kerambit', 'keramboja', 'keramik', 'keramikus', 'kerampagi', 'kerampang', 'keramunting', 'keran', 'kerancang', 'keranda', 'kerang', 'kerangas', 'kerangkai', 'kerangkeng', 'kerani', 'keranjang', 'keranjat', 'keranji', 'keranta', 'kerantong', 'kerap', 'kerapu', 'keras', 'kerasan', 'kerat', 'keratabasa', 'keratin', 'keratitis', 'keratoelastin', 'keraton', 'kerau', 'kerawai', 'kerawak', 'kerawang', 'kerawat', 'kerawit', 'kerbang', 'kerbat', 'kerbau', 'kerbuk', 'kercing', 'kercit', 'kercut', 'kerdak', 'kerdam', 'kerdil', 'kerdom', 'kere', 'kerebok', 'kereceng', 'kerecik', 'keredak', 'keredep', 'keredok', 'keredong', 'kerek', 'kereket', 'kerekot', 'kerekut', 'keremi', 'keremot', 'kerempeng', 'kerempung', 'keremus', 'keren', 'kerencang', 'kerencung', 'kerendang', 'kereneng', 'kereng', 'kerengga', 'kerengkam', 'kerentam', 'kerentang', 'kerenting', 'kerenyam', 'kerenyot', 'kerepek', 'kerepes', 'kerepot', 'kerepyak', 'kerese pese', 'keresek', 'kereseng', 'keresot', 'kereta', 'kereta api', 'keretan', 'keretek', 'keretot', 'keretut', 'kereweng', 'keri', 'keriap', 'kerias', 'keriau', 'kerical', 'kericau', 'keridas', 'keridik', 'kerih', 'kerikal', 'kerikam', 'kerikil', 'kerikit', 'kerimut', 'kerinan', 'kerincing', 'kerinding', 'kering', 'keringat', 'keriningan', 'kerinjal', 'kerinjang', 'kerinjing', 'kerintil', 'kerinting', 'kerip', 'keripik', 'keriput', 'keris', 'kerisi', 'kerisik', 'kerising', 'kerisut', 'kerit', 'keritik', 'keriting', 'keriuk', 'keriut', 'kerja', 'kerjang', 'kerjantara', 'kerjap', 'kerkah', 'kerkak', 'kerkap', 'kerkau', 'kerkop', 'kerkup', 'kerlap', 'kerling', 'kerlip', 'kermak', 'kermanici', 'kermi', 'kernai', 'kerneli', 'kernet', 'kernu', 'kernyau', 'kernyih', 'kernying', 'kernyit', 'kernyut', 'kero', 'kerobak', 'kerobat', 'kerobek', 'keroco', 'kerocok', 'kerogen', 'keroh', 'kerok', 'kerokot', 'keromong', 'keron', 'keroncang', 'keroncong', 'keroncor', 'kerong', 'kerong kerang', 'kerongsang', 'kerontang', 'kerop', 'keropak', 'keropeng', 'keropok', 'keropos', 'kerosak', 'kerosek', 'kerosi', 'kerosin', 'kerosok', 'kerosong', 'kerot', 'kerotak', 'kerotot', 'keroyok', 'kerpai', 'kerpak', 'kerpas', 'kerpubesi', 'kerpuk', 'kerpus', 'kers', 'kersai', 'kersak', 'kersang', 'kersani', 'kersen', 'kersik', 'kersuk', 'kertaaji', 'kertah', 'kertak', 'kertang', 'kertap', 'kertas', 'kertau', 'kertuk', 'kertus', 'keruan', 'kerubin', 'kerubung', 'kerubut', 'kerucil', 'kerucut', 'kerudung', 'keruh', 'keruh bumi', 'keruit', 'keruk', 'kerukut', 'kerul', 'keruma', 'kerumit', 'kerumuk', 'kerumun', 'kerumus', 'kerun', 'kerung', 'kerunkel', 'kerunting', 'keruntung', 'kerunyut', 'kerup', 'kerupuk', 'kerut', 'kerutak', 'kerutup', 'keruyuk', 'kes', 'kesah', 'kesak', 'kesal', 'kesam', 'kesambet', 'kesambi', 'kesan', 'kesang', 'kesat', 'kesatria', 'kesek', 'kesel', 'keseleo', 'kesemek', 'keseran', 'keseser', 'keset', 'kesi', 'kesiap', 'kesik', 'kesima', 'kesimbukan', 'kesip', 'kesiur', 'keskul', 'kesomplok', 'kesongo', 'kesot', 'kesrakat', 'kesting', 'kesturi', 'kesuma', 'kesumat', 'kesumba', 'kesup', 'kesusu', 'kesut', 'keta', 'ketaban', 'ketai', 'ketak', 'ketakong', 'ketal', 'ketam', 'ketambak', 'ketampi', 'ketan', 'ketang', 'ketap', 'ketapak', 'ketapang', 'ketapek', 'ketar', 'ketarap', 'ketat', 'ketaton', 'ketaya', 'ketayap', 'ketegar', 'ketek', 'ketel', 'ketela', 'ketemu', 'ketena', 'keteng', 'ketepel', 'ketepeng', 'keter', 'ketes', 'keteter', 'ketgat', 'keti', 'ketiak', 'ketial', 'ketiap', 'ketiau', 'ketiban', 'ketiding', 'ketik', 'ketika', 'ketil', 'ketilang', 'ketimbis', 'ketimbul', 'keting', 'ketinjau', 'ketinting', 'ketip', 'ketiplak', 'ketipung', 'ketirah', 'ketis', 'ketitir', 'ketogenesis', 'ketok', 'ketola', 'ketombe', 'ketonemia', 'ketonggeng', 'ketonuria', 'ketopong', 'ketoprak', 'ketosa', 'ketrek', 'ketu', 'ketua', 'ketuat', 'ketuban', 'ketuir', 'ketuk', 'ketul', 'ketumbar', 'ketumbi', 'ketumbit', 'ketumbu', 'ketumpang', 'ketun', 'ketungging', 'ketup', 'ketupa', 'ketupat', 'ketupuk', 'ketur', 'ketus', 'kevin']

In [34]:
# grammar rules before applying POS tags
grammar = [
	{
		"pattern":r"(?<!\w)di\w+",
		"pesan":"'di' harus pisah dengan kata benda/tempat.",
        "koreksi":"",  # 
        "contoh":"di kantor",
        "status":"error1",
        "pengecualian":di,
        "err_id":1
	},
    {
		"pattern":r"^(ke\w+)",
		"pesan":"'ke' harus pisah dengan kata yang mengikutinya.",
        "koreksi":"",  # 
        "contoh":"ke sana",
        "status":"error1",
        "pengecualian":ke,
        "err_id":2
	},
    {
		"pattern":r"\w+pun",
		"pesan":"Partikel 'pun' harus dipisah dengan kata sebelumnya.",
        "koreksi":"",  # 
        "contoh":"makanan pun",
        "status":"error1",
        "pengecualian":["meskipun", "adapun", "bagaimanapun", "walaupun", "ampun"],
        "err_id":3
	}
]

In [35]:
def word_checker(testing):
    # iterate through each word in a sentence
    for id, sentence in enumerate(testing):
        # iterate through possible errors
        for err in grammar:
            found = re.search(err["pattern"], sentence)
            if found:

                # error "di" dan "ke"
                if (err["err_id"] == 1 or err["err_id"] == 2) and \
                (testing[id] not in err["pengecualian"]):
                    kata = testing[id]
                    k1 = kata[:2]  # di
                    k2 = kata[2:]  # rumah
                    koreksi = k1, k2
                    testing[id:id+1] = (koreksi)
                    grammar_id.extend([id, id+1])
                
                # error "pun"
                if err["err_id"] == 3 and \
                (testing[id] not in err["pengecualian"]):
                    kata = testing[id]
                    k1 = kata[:-3]  # makanan
                    k2 = kata[-3:]  # pun
                    koreksi = k1, k2
                    testing[id:id+1] = (koreksi)
                    grammar_id.append(id)

## Setelah POS tag

Sekarang input kalimat yang sudah diedit ke model POS tagger

In [36]:
# grammar rules
grammar_pos = [
	{
		"pattern":["VB", "PRP", "NN"],
		"pesan":"Struktur kata kerja + saya/mereka/dia + kata benda tidak sesuai.",
        "koreksi":[0,2,1],  # VB NN PRP
        "contoh":"memakan apel saya",
        "status":"error1",
        "pengecualian":["memberi"],
        "err_id":4
	},    
    {
		"pattern":["JJ", "NN"],
		"pesan":"Kata sifat ditempatkan setelah kata benda.",
        "koreksi":[1, 0],  # NN JJ
        "contoh":"mobil merah",
        "status":"error1",
        "pengecualian":["besok", "hari", "kemarin", "pagi", "saat"],
        "err_id":7
	},
    {
		"pattern":["IN", "VB"],
		"pesan":"'di' harus digabung dengan kata kerja.",
        "koreksi":"VB",  # replace "IN", "VB" with "VB"
        "contoh":"dimakan",
        "status":"error1",
        "pengecualian":[],
        "err_id":5
	},
    {
		"pattern":["PRP", "NN"],
		"pesan":"Struktur saya/mereka/dia diikuti kata benda tidak sesuai",
        "koreksi":[1, 0],  # NN JJ
        "contoh":"meja saya",
        "status":"error1",
        "pengecualian":["memberi"],
        "err_id":8
	},
    {
		"pattern":["PRP", "VB", "WH"],
		"pesan":"Kalimat tanya diakhiri dengan tanda tanya",
        "koreksi":"Z",  # add "?"
        "contoh":"kamu makan apa?",
        "status":"error3",
        "pengecualian":[],
        "err_id":16
	},
    {
		"pattern":["VB"],
		"pesan":"Kata sangat atau agak tidak bisa digabung dengan kata kerja.",
        "koreksi":"VB",  # replace "sangat/agak", "VB" with "VB"
        "contoh":"sangat makan",
        "status":"error1",
        "pengecualian":["membahayakan", "berbahaya", "berbisa", "kecewa", \
                        "mengecewakan", "suka", "menyukai", "berharap", \
                        "mengharapkan", "membenci", "berjasa"],
        "err_id":6
	},
    {
		"pattern":["VB", "NEG"],
		"pesan":"Struktur yang benar adalah kata negasi+kata kerja",
        "koreksi":[1, 0],  # NEG VB
        "contoh":"tidak makan",
        "status":"error1",
        "pengecualian":[],
        "err_id":9
	},
    {
		"pattern":["VB", "RB"],
		"pesan":"Kata kerja tidak bisa digabung dengan kata 'sekali'.",
        "koreksi":"VB",  # replace "VB", "sekali" with "VB"
        "contoh":"makan sekali",
        "status":"error1",
        "pengecualian":["membahayakan", "berbahaya", "berbisa", "kecewa", \
                        "mengecewakan", "suka", "menyukai", "berharap", \
                        "mengharapkan", "membenci", "berjasa"],
        "err_id":10
	},
    {
		"pattern":["RB", "JJ", "RB"],
		"pesan":"Struktur kata sangat+kata sifat+sekali tidak efektif.",
        "koreksi":"",  # remove "sekali"
        "contoh":"sangat indah sekali",
        "status":"error2",
        "pengecualian":[],
        "err_id":11
	},
    {
		"pattern":["SC", "SC"],
		"pesan":"Struktur kata agar+supaya tidak efektif",
        "koreksi":"",  # remove "agar"
        "contoh":"Andi belajar agar lulus",
        "status":"error2",
        "pengecualian":[],
        "err_id":12
	},
    {
		"pattern":["IN", "SC"],
		"pesan":"Struktur kata demi+untuk tidak efektif",
        "koreksi":"",  # remove "demi"
        "contoh":"dia bekerja untuk kebutuhan hidup",
        "status":"error2",
        "pengecualian":[],
        "err_id":13
	},
    {
		"pattern":["CD", "VB", "CD"],
		"pesan":"Struktur kata banyak+terdapat+berbagai tidak efektif",
        "koreksi":"",  # remove "banyak"
        "contoh":"terdapat berbagai jenis tanaman",
        "status":"error2",
        "pengecualian":[],
        "err_id":14
	},
    {
		"pattern":["DT", "NN", "CD"],
		"pesan":"Struktur kata para+kata benda+semua tidak efektif",
        "koreksi":"",  # remove "para"
        "contoh":"para hadirin semua",
        "status":"error2",
        "pengecualian":[],
        "err_id":15
	},
    {
		"pattern":["VB", "RP"],
		"pesan":"Kata kerja digabung dengan partikel 'lah'",
        "koreksi":"VB",  # replace "IN", "VB" with "VB"
        "contoh":"makanlah",
        "status":"error1",
        "pengecualian":[],
        "err_id":17
	},
    {
		"pattern":["NNP"],
		"pesan":"Gunakan huruf besar diawal kata",
        "koreksi":"",  # capitalize the first letter
        "contoh":"Budi",
        "status":"error4",
        "pengecualian":[],
        "err_id":18
	},
    {
		"pattern":["WH"],
		"pesan":"Kalimat tanya diakhiri dengan tanda tanya",
        "koreksi":"",  # add "?" or change "." to "?"
        "contoh":"siapa kalian?",
        "status":"error3",
        "pengecualian":[],
        "err_id":19
	},
    {
		"pattern":["SC"],
		"pesan":"Kalimat tanya diakhiri dengan tanda tanya",
        "koreksi":"",  # add "?" or change "." to "?"
        "contoh":"siapa kalian?",
        "status":"error3",
        "pengecualian":[],
        "err_id":19
	},
    {
		"pattern":["SYM", "CD"],
		"pesan":"Nilai mata uang disambung dengan nominal",
        "koreksi":"CD",  # 
        "contoh":"Rp100",
        "status":"error4",
        "pengecualian":[],
        "err_id":20
	},
    {
		"pattern":["NN"],
		"pesan":"'jam' diganti dengan 'pukul'",
        "koreksi":"",  # 
        "contoh":"pukul 6 sore",
        "status":"error4",
        "pengecualian":[],
        "err_id":21
	},
    {
		"pattern":["IN"],
		"pesan":"Struktur kata mundur+ke+belakang tidak efektif",
        "koreksi":"",  # remove "ke belakang"
        "contoh":"dia mundur ke belakang",
        "status":"error2",
        "pengecualian":[],
        "err_id":22
	},
    {
		"pattern":["CC"],
		"pesan":"Gunakan tanda koma sebelum kata 'dan'",
        "koreksi":"",  # ..., ..., dan ...
        "contoh":"ayah, ibu, dan adik",
        "status":"error3",
        "pengecualian":[],
        "err_id":23
	},
]

In [37]:
def grammar_checker(testing, testing_list, testing_pos):

    check = 0

    # iterate through possible errors
    for err in grammar_pos:
        indexes = []

        # look for error patterns in the list of POS tags
        for i in range(len(testing_pos)):

            # i += check

            if testing_pos[i:i+len(err["pattern"])] == err["pattern"]:  # found a match
                indexes.append((i, i+len(err["pattern"])))

                # error VB PRP NN
                if err["err_id"] == 4 and \
                (testing_list[i] not in err["pengecualian"]):
                    koreksi_pos = [testing_pos[i:i+len(err["pattern"])][a] for a in err["koreksi"]]
                    koreksi = [testing_list[i:i+len(err["pattern"])][a] for a in err["koreksi"]]

                    for j in range(len(indexes)):
                        testing_list[i:indexes[j][1]] = koreksi
                        testing_pos[indexes[j][0]:indexes[j][1]] = koreksi_pos
                    
                    grammar_id.extend(range(i, i+len(err["pattern"])))

                # error "di" + kata kerja
                if err["err_id"] == 5 and testing_list[i] == "di":
                    
                    for j in range(len(indexes)):
                        k1 = testing_list[indexes[j][0]]
                        k2 = testing_list[indexes[j][0]+1]
                        testing_list[indexes[j][0]] = k1 + k2
                        testing_list.pop(indexes[j][0]+1)
                        testing_pos.pop(i)
                        grammar_id.extend([indexes[j][0], indexes[j][0]+1])
                        # check += 1

                # error "sangat"/"agak" + kata kerja
                if err["err_id"] == 6 and (i != 0):

                    if (testing_list[i-1] == "sangat" or testing_list[i-1] == "agak") \
                    and (testing_list[i] not in err["pengecualian"]):
                        testing_list.pop(i-1)
                        testing_pos.pop(i-1)
                        grammar_id.extend([i-1, i])
                        # check -= 1


                    # error PRP/NN + VB + "sedang"
                    if i != len(testing_list)-1:
                        if ((testing_pos[i-1] == "PRP") or (testing_pos[i-1] == "NN")) and \
                        (testing_list[i+1] == "sedang"):
                            testing_list[i], testing_list[i+1] = testing_list[i+1], testing_list[i]
                            testing_pos[i], testing_pos[i+1] = testing_pos[i+1], testing_pos[i]
                            grammar_id.extend([i-1, i, i+1])

                # error JJ NN
                if err["err_id"] == 7 and \
                (testing_list[i+1] not in err["pengecualian"]):
                    koreksi_pos = [testing_pos[i:i+len(err["pattern"])][a] for a in err["koreksi"]]
                    koreksi = [testing_list[i:i+len(err["pattern"])][a] for a in err["koreksi"]]

                    for j in range(len(indexes)):
                        testing_list[i:indexes[j][1]] = koreksi
                        testing_pos[indexes[j][0]:indexes[j][1]] = koreksi_pos
                        grammar_id.extend([i, i+1])

                # error PRP NN
                if err["err_id"] == 8:
                    if i != 0:
                        if testing_pos[i-1] == "VB":
                            continue

                    koreksi_pos = [testing_pos[i:i+len(err["pattern"])][a] for a in err["koreksi"]]
                    koreksi = [testing_list[i:i+len(err["pattern"])][a] for a in err["koreksi"]]

                    for j in range(len(indexes)):
                        testing_list[i:indexes[j][1]] = koreksi
                        testing_pos[indexes[j][0]:indexes[j][1]] = koreksi_pos
                        grammar_id.extend([i, i+1])

                # error VB NEG
                if err["err_id"] == 9:
                    koreksi_pos = [testing_pos[i:i+len(err["pattern"])][a] for a in err["koreksi"]]
                    koreksi = [testing_list[i:i+len(err["pattern"])][a] for a in err["koreksi"]]

                    for j in range(len(indexes)):
                        testing_list[i:indexes[j][1]] = koreksi
                        testing_pos[indexes[j][0]:indexes[j][1]] = koreksi_pos
                        grammar_id.extend([i, i+1])

                # error kata kerja + "sekali"
                if err["err_id"] == 10 and (testing_list[i+1] == "sekali") and \
                (testing_list[i] not in err["pengecualian"]):
                    testing_list.pop(i+1)
                    testing_pos.pop(i+1)
                    grammar_id.extend([i, i+1])
                    # check -= 1
                
                # error "sangat" + kata sifat + "sekali"
                if err["err_id"] == 11 and (testing_list[i+2] == "sekali") and \
                (testing_list[i] == "sangat"):
                    testing_list.pop(i+2)
                    testing_pos.pop(i+2)
                    efektif_id.extend([i, i+2, i+3])
                
                # error "agar" + "supaya"
                if err["err_id"] == 12 and (testing_list[i] == "agar") and \
                (testing_list[i+1] == "supaya"):
                    testing_list.pop(i+1)
                    testing_pos.pop(i+1)
                    efektif_id.extend([i, i+1])

                # error "demi" + "untuk"
                if err["err_id"] == 13 and (testing_list[i] == "demi") and \
                (testing_list[i+1] == "untuk"):
                    testing_list.pop(i)
                    testing_pos.pop(i)
                    efektif_id.extend([i, i+1])
                
                # error "banyak" + "terdapat" + "berbagai"
                if err["err_id"] == 14 and (testing_list[i] == "banyak") and \
                (testing_list[i+1] == "terdapat") and \
                (testing_list[i+2] == "berbagai"):
                    testing_list.pop(i)
                    testing_pos.pop(i)
                    efektif_id.extend(range(i, i+3))

                # error "para" + kata benda + "semua"
                if err["err_id"] == 15 and (testing_list[i] == "para") and \
                (testing_list[i+2] == "semua"):
                    testing_list.pop(i)
                    testing_pos.pop(i)
                    efektif_id.extend([i, i+1])
                
                # error tanda tanya
                if err["err_id"] == 16:
                    # end of sentence
                    if i+len(err["pattern"]) >= len(testing_list):
                        testing_list.append("?")
                        testing_pos.append("Z")
                        punc_id.extend(range(i, i+len(err["pattern"])))

                    elif testing_list[i+3] != "?":
                        testing_list.insert(i+len(err["pattern"]), "?")
                        testing_pos.insert(i+len(err["pattern"]), "Z")
                        punc_id.extend(range(i, i+len(err["pattern"])))

                # error kata kerja + "lah"
                if err["err_id"] == 17 and testing_list[i+1] == "lah":
                    
                    for j in range(len(indexes)):
                        k1 = testing_list[indexes[j][0]]
                        k2 = testing_list[indexes[j][0]+1]
                        testing_list[indexes[j][0]] = k1 + k2
                        testing_list.pop(indexes[j][0]+1)
                        testing_pos.pop(i)
                        grammar_id.extend([i, i+1])

                # error huruf kapital
                if err["err_id"] == 18:
                    vowels = ['a', 'e', 'i', 'o', 'u', 'A', 'E', 'I', 'O', 'U']
                    if testing_list[i] != testing_list[i].capitalize():
                        testing_list[i] = testing_list[i].capitalize()
                        if (len([j for j in testing_list[i] if j in vowels]) == 0) \
                        or (len([j for j in testing_list[i] if j in vowels]) == len(testing_list[i])):
                            testing_list[i] = testing_list[i].upper()
                        kapital_id.append(i)
                
                # error tanda tanya part 2
                if err["err_id"] == 19 and ((i == 0) or (testing_pos[i-1] == "Z")):

                    if (testing_pos[i] == "WH") or ((testing_pos[i] == "SC") \
                    and (testing_list[i] in ["siapa", "bagaimana", "apa", \
                                          "kapan", "di mana", "berapa", \
                                          "mengapa", "mengapa"])):
                        z_id = [j for j, x in enumerate(testing_pos[i+1:]) if x == "Z"]
                        # no punctuation after the question
                        if not z_id:
                            testing_list.append("?")
                            testing_pos.append("Z")
                            punc_id.append(len(kalimat)-1)
                        
                        # punctuation isn't a question mark
                        else:
                            # if testing_list[i+z_id[0]+1] != "?":
                            #     if testing_list[i+z_id[1]+1] in ["'", '"', ")", "]", "}"]:
                            #         k1 = "?"
                            #         k2 = 
                            #         testing_
                            #     else:
                            #         testing_list[i+z_id[0]+1] = "?"
                            #         testing_pos[i+z_id[0]+1] = "Z"
                            #     punc_id.append(i+z_id[0]+1)
                          
                            if testing_list[i+z_id[0]+1] != "?":
                                if testing_list[i+z_id[0]+1] in ["'", '"', ")", "]", "}"]:
                                    testing_list.insert(i+z_id[0]+1, "?")
                                    testing_pos.insert(i+z_id[0]+1, "Z")
                                else:
                                    testing_list[i+z_id[0]+1] = "?"
                                    testing_pos[i+z_id[0]+1] = "Z"
                                punc_id.append(i+z_id[0]+1)

                # error "Rp" + nominal
                if err["err_id"] == 20 and testing_list[i] in ["Rp", "rp", "RP"]:
                    
                    for j in range(len(indexes)):
                        k1 = testing_list[indexes[j][0]]
                        k2 = testing_list[indexes[j][0]+1]
                        testing_list[indexes[j][0]] = k1 + k2
                        testing_list.pop(indexes[j][0]+1)
                        testing_pos.pop(i)
                        grammar_id.extend([i, i+1])
                
                # error "pukul" + angka
                if err["err_id"] == 21 and testing_list[i] in ["jam", "pukul"] \
                and testing_list[i+1].isnumeric() and i != len(testing_list):

                    if testing_list[i] == "jam":
                        testing_list[i] = "pukul"
                        grammar_id.append(i)

                    if i+1 != len(testing_list)-1:
                        z_id = [j for j, x in enumerate(testing_pos[i+1:]) if x == "Z"]
                        # error ":" 
                        if (z_id[0] == 1):
                            if (testing_list[i+2] == ":") and \
                            (testing_list[i+3].isnumeric()):
                                k1 = testing_list[i+1]
                                k2 = "."
                                k3 = testing_list[i+3]
                                testing_list[i+1] = k1 + k2 + k3
                                testing_list.pop(i+2)
                                testing_list.pop(i+2)
                                testing_pos.pop(i+2)
                                testing_pos.pop(i+2)
                                grammar_id.extend(range(i+1, i+4))
                
                # error "maju/mundur/naik" + "ke" + "depan/belakang/atas"
                if err["err_id"] == 22 and i != 0 and i != len(testing_list)-1:
                    if (testing_list[i-1] in ["naik", "mundur", "maju"]) and \
                    (testing_list[i] == "ke") and \
                    (testing_list[i+1] in ["atas", "belakang", "depan"]):
                        testing_list.pop(i)
                        testing_pos.pop(i)
                        testing_list.pop(i)
                        testing_pos.pop(i)
                        efektif_id.extend(range(i-1, i+2))

                # error koma sebelum "dan"
                if err["err_id"] == 23 and testing_list[i] == "dan" and i-3 >= 0:
                    if testing_list[i-2] == ",":
                        testing_list.insert(i, ",")
                        testing_pos.insert(i, "Z")
                        punc_id.append(i)
                        

                testing = " ".join(testing_list)



In [38]:
def finishing_touch(testing_list, testing_pos, kapital_id):
    # capitalize the first word
    if testing_list[0] != testing_list[0].capitalize():
        kapital_id.append(0)
    testing_list[0] = testing_list[0].capitalize()

    # end the sentence with a dot
    if (testing_list[-1] != ".") and (testing_list[-1] != "?"):
        testing_list.append(".")
        testing_pos.append("Z")
        punc_id.append(len(kalimat)-1)

    z_id = [i for i, x in enumerate(testing_pos) if x == "Z"]

    # capitalize the first word after a dot or a question mark
    for i in z_id[:-1]:
        if testing_list[i] in [".", "?", "!", "'", '"']:
            testing_list[i+1] = testing_list[i+1].capitalize()

    # get kapital_id
    z_id_1 = [i for i, x in enumerate(kalimat[:-1]) if x in [".", "?", "!", "'", '"']]
    for i in z_id_1:
        if kalimat[i+1] != kalimat[i+1].capitalize():
                    kapital_id.append(i+1)

    check = 0
    comma = 0

    for i in z_id:

        sym_count = 0
        # count previous quotation marks
        for kata in testing_list[:i-3]:
            for a in kata:
                if a in ["'", '"']:
                    sym_count += 1

        i += check

        try:
            if i != len(testing_list)-1 and i < len(testing_list):

                # merge " with the next word
                if (sym_count % 2 == 0) and \
                (testing_list[i] in ["'", '"']):
                    k1 = testing_list[i]
                    k2 = testing_list[i+1]
                    testing_list[i] = k1 + k2
                    testing_list.pop(i+1)
                    testing_pos.pop(i)
                    check -= 1
                
                    # no punctuation before ""                
                    if (testing_pos[i-1] != "Z") and \
                    (testing_list[i-1][-1] not in [",", ".", "?", "!"]):
                        testing_list.insert(i, ",")
                        k1 = testing_list[i-1]
                        k2 = testing_list[i]
                        testing_list[i-1] = k1 + k2
                        testing_list.pop(i)
                        testing_pos.pop(i)
                        check -= 1

                # merge " with the previous word (comma before "")
                elif (testing_list[i] in [",", ".", "?", "!"]) and \
                (testing_list[i+1] in ["'", '"']):
                    k1 = testing_list[i-1]
                    k2 = testing_list[i]
                    testing_list[i-1] = k1 + k2
                    testing_list.pop(i)
                    testing_pos.pop(i)
                    check -= 1

                # merge " with the previous word (normal sentence)
                elif i != 0 and (testing_list[i] not in ["-", "("]):
                    k1 = testing_list[i-1]
                    k2 = testing_list[i]
                    testing_list[i-1] = k1 + k2
                    testing_list.pop(i)
                    testing_pos.pop(i)
                    check -= 1

                # brackets/parentheses
                elif testing_list[i] in ["("]:
                    k1 = testing_list[i]
                    k2 = testing_list[i+1]
                    testing_list[i] = k1 + k2
                    testing_list.pop(i+1)
                    testing_pos.pop(i)
                    check -= 1

                # plural nouns with "-"
                if (i != 0) and (i != len(testing_list)-1) and \
                (testing_list[i] == "-"):
                    k1 = testing_list[i-1]
                    k2 = testing_list[i]
                    k3 = testing_list[i+1]
                    testing_list[i-1] = k1 + k2 + k3
                    testing_list.pop(i)
                    testing_pos.pop(i)
                    testing_list.pop(i)
                    testing_pos.pop(i)
                    check -= 2
            
            else:
                k1 = testing_list[i-1]
                k2 = testing_list[i]
                testing_list[i-1] = k1 + k2
                testing_list.pop(i)
                testing_pos.pop(i)
                check -= 1

        except Exception as e:
            print(e)

    return " ".join(testing_list), kapital_id

# EsAI.id

In [40]:
kalimat = "dia beertanya, 'anda makan apa' Dia sedang sangat makn saya apel dikantor. saya naik ke atas"
print(f"Original text:  {kalimat}")
print("="*150)

punc_id = []
efektif_id = []
grammar_id = []
kapital_id = []

# preprocessing
kalimat = split_kalimat(kalimat)

# typo checker
kalimat_baru = []
typos_id = []
for id, kata in enumerate(kalimat):
    if (kata.isalpha()) and ((kata != "Rp") or (kata.isnumeric())):
        kalimat_baru.append(correction(kata.lower()))
        if correction(kata).lower() != kata.lower():
            typos_id.append(id)
    else:
        kalimat_baru.append(kata)
print(f"1   Typo correction:    {kalimat_baru}")

# rule-based word/typo checker
word_checker(kalimat_baru)
print(f"2   Word checker:       {kalimat_baru}")

# pos tagger
pos_kalimat = prediksi_pos(kalimat_baru)
print(f"3   POS tags:           {pos_kalimat}")

# rule-based grammar checker
grammar_checker(" ".join(kalimat_baru), kalimat_baru, pos_kalimat)
print(f"4   Grammar checker:    {kalimat_baru}")

# capitalization + finishing touches
kalimat_final, kapital_id = finishing_touch(kalimat_baru, pos_kalimat, kapital_id)

# typos etc.
print("="*150)
print("Detected errors:")
print(f"1   Typos:              {sorted(typos_id)}")
print(f"2   Huruf kapital:      {sorted(kapital_id)}")
print(f"3   Tanda baca:         {sorted(punc_id)}")
print(f"4   Tidak efektif:      {sorted(efektif_id)}")
print(f"5   Grammar/lainnya:    {sorted(set(grammar_id))}")

print("="*150)
print(f"Corrected text: {kalimat_final}")

Original text:  dia beertanya, 'anda makan apa' Dia sedang sangat makn saya apel dikantor. saya naik ke atas
1   Typo correction:    ['dia', 'bertanya', ',', "'", 'anda', 'makan', 'apa', "'", 'dia', 'sedang', 'sangat', 'makan', 'saya', 'apel', 'dikantor', '.', 'saya', 'naik', 'ke', 'atas']
2   Word checker:       ['dia', 'bertanya', ',', "'", 'anda', 'makan', 'apa', "'", 'dia', 'sedang', 'sangat', 'makan', 'saya', 'apel', 'di', 'kantor', '.', 'saya', 'naik', 'ke', 'atas']
3   POS tags:           ['PRP', 'VB', 'Z', 'Z', 'PRP', 'VB', 'WH', 'Z', 'PRP', 'MD', 'RB', 'VB', 'PRP', 'NN', 'IN', 'NN', 'Z', 'PRP', 'VB', 'IN', 'IN']
4   Grammar checker:    ['dia', 'bertanya', ',', "'", 'anda', 'makan', 'apa', '?', "'", 'dia', 'sedang', 'makan', 'apel', 'saya', 'di', 'kantor', '.', 'saya', 'naik']
Detected errors:
1   Typos:              [1, 11]
2   Huruf kapital:      [0, 4, 16]
3   Tanda baca:         [4, 5, 6, 19]
4   Tidak efektif:      [18, 19, 20]
5   Grammar/lainnya:    [11, 12, 13, 14, 15]
